In [6]:
import json
import polars as pl
import re

In [7]:
data_dir = '/Volumes/PortableSSD/CSS/data/processed/'

#### Get metaphor matched posts through FrameBERT

In [8]:
data_df = pl.read_csv(data_dir+'posts_2015-21_ps_min_2c_politics.csv')
with open(data_dir+'post_cluster_matches_5.json') as f:
    data = json.load(f)

##### load metaphors. get all unigram metaphors

In [9]:
NO_META_LIST = ['out', 'up', 'tip', 'check']

In [10]:
with open(data_dir+'meta_dict_full.json', 'r') as fp:
        data = json.load(fp)
meta_list = []
for key, values in data.items():
    meta_list.extend(values)
# remove duplicates
meta_list = list(set(meta_list))
# filter metaphors
print('filtering metaphors')
meta_list = [meta.replace("'", '') for meta in meta_list]
meta_list = [re.sub(r"[^a-zA-Z0-9]+", ' ', meta).lower() for meta in meta_list]
meta_list = [m for m in meta_list if m not in NO_META_LIST]

filtering metaphors


In [16]:
uni_metas = []

for meta in meta_list:
    if len(meta.split()) == 1:
        uni_metas.append(meta)

##### After running throught FrameBERT

In [228]:
meta_res_df = pl.read_csv('/users/ujan/MetaphorFrame/2017-02-14_53_predictions.tsv', separator='\t')

In [229]:
meta_res_df.drop_nulls().head(20)

Tokens,Borderline_metaphor,Real_metaphors,Frame_label
str,i64,i64,str
"""donald""",0,0,"""_"""
"""trump""",0,0,"""_"""
"""ally""",0,0,"""Personal_relat…"
"""says""",0,0,"""Statement"""
"""reince""",0,0,"""_"""
…,…,…,…
"""in""",1,0,"""_"""
"""over""",1,0,"""_"""
"""his""",0,0,"""_"""


In [230]:
meta_res_df = meta_res_df.drop_nulls()

In [231]:
len(meta_res_df)

11

##### Get FrameBERT metaphor matches (true positives)

In [232]:
tokens = meta_res_df['Tokens'].to_list()
metas = meta_res_df['Real_metaphors'].to_list()
frames = meta_res_df['Frame_label'].to_list()

In [233]:
sent = []
meta_words = []
frame_labels = []
data = []

for t in range(len(tokens)):
    if tokens[t] is None:
        continue
    sent.append(tokens[t])
    if metas[t] == 1 and tokens[t] in uni_metas:
        meta_words.append(tokens[t])
        frame_labels.append(frames[t])
    if(tokens[t] =='.'):
        if len(sent) > 0 and len(meta_words) > 0:
            data.append((' '.join(sent), meta_words, frame_labels))
        sent = []
        meta_words = []
        frame_labels = []

In [234]:
len(data)

0

In [159]:
# 2015-05-16_17

data[0]

('jeb bush 3 days 4 different answers about iraq .', ['bush'], ['_'])

In [44]:
# 2015-05-16_14

data[0]

('undiginified bush camp and other neocon oriented gop presidential contenders are now belly crawling before crass and kitsch zionist casino billionaire sheldon adelson in the hope hell throw some gambling business proceeds their way .',
 ['bush'],
 ['_'])

In [206]:
# 2017-02-14_30

data[0]

('reince priebus staging a coup against general flynn standwithflynn .',
 ['coup'],
 ['Change_of_leadership'])

##### Get FrameBERT non metaphor matches (false positives)

In [235]:
sent = []
meta_words = []
frame_labels = []
data = []

for t in range(len(tokens)):
    if tokens[t] is None:
        continue
    sent.append(tokens[t])
    if metas[t] == 0 and tokens[t] in uni_metas:
        meta_words.append(tokens[t])
        frame_labels.append(frames[t])
    if(tokens[t] =='.'):
        if len(sent) > 0 and len(meta_words) > 0:
            data.append((' '.join(sent), meta_words, frame_labels))
        sent = []
        meta_words = []
        frame_labels = []

In [236]:
len(data)

0

In [69]:
# 2015-05-16_24

data[0]

('after jeb bush flap a look at hillarys troubled iraq war history .',
 ['bush'],
 ['_'])

In [79]:
2015-05-16_31

data[0]

('ivy ziedrich college student warms to role as jeb bush critic on isis .',
 ['bush'],
 ['_'])

In [80]:
2015-05-16_31

data[1]

('student who told jeb bush your brother created isis speaks out about incident ivy ziedrich challenged the likely presidential candidate after he blamed the militant group s formation on barack obama for withdrawing troops from iraq .',
 ['bush'],
 ['_'])

In [122]:
# 2017-02-04_158

data[0]

('rt michaelkeyes freakin nfl built stadium walls to keep fans that havnt paid for a ticket from entering the game notmysuperbowl ten gop .',
 ['game'],
 ['_'])

#### Running FrameBERT on non metaphor matched (our algo) samples

In [79]:
data_df = pl.read_csv(data_dir+'politics_sample.csv')
data_df = data_df.sample(fraction=1.0, shuffle=True, seed=42).drop_nulls()

comments = [comment.replace("'", '') for comment in data_df['body'].to_list()]
comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in comments]
ids = data_df['id'].to_list()

comments_long = []
ids_long = []
# filter by char 
for c in range(len(comments)):
    if len(comments[c]) >= 150:
        comments_long.append(comments[c])
        ids_long.append(ids[c])

comments = comments_long[:10000]
ids = ids_long[:10000]

In [80]:
data_df = data_df.filter(pl.col('id').is_in(ids))

exact_ids = []
for key, matches in exact_matches.items():
    if len(matches[1]) > 0:
        exact_ids.extend(matches[1])

non_meta_list = data_df.filter(~pl.col('id').is_in(exact_ids))['body'].to_list()
non_meta_list = non_meta_list[:1000]

##### After running throught FrameBERT

In [112]:
non_meta_res_df = pl.read_csv(data_dir+'non_meta_predictions.csv')
non_meta_res_df.head()

Tokens,Borderline_metaphor,Real_metaphors,Frame_label
str,i64,i64,str
"""Yes""",0,0,"""_"""
""",""",0,0,"""_"""
"""he""",0,0,"""_"""
"""would""",0,0,"""_"""
"""“""",0,0,"""_"""


##### Get FrameBERT metaphor matches

In [113]:
tokens = non_meta_res_df['Tokens'].to_list()
metas = non_meta_res_df['Real_metaphors'].to_list()
#metas = non_meta_res_df['Borderline_metaphor'].to_list()
frames = non_meta_res_df['Frame_label'].to_list()

In [114]:
sent = []
meta_words = []
frame_labels = []
data = []

for t in range(len(tokens)):
    if tokens[t] is None:
        continue
    sent.append(tokens[t])
    if metas[t] == 1:
        meta_words.append(tokens[t])
        frame_labels.append(frames[t])
    if(tokens[t] =='.'):
        if len(sent) > 0 and len(meta_words) > 0:
            data.append((' '.join(sent), meta_words, frame_labels))
        sent = []
        meta_words = []
        frame_labels = []

In [118]:
data[0]

('Yes , he would “ generally ” agree it impeded .', ['impeded'], ['Hindering'])

In [119]:
data[1]

("Why ? Because you say so ? I 'll take the President and CEO of a multi-billion dollar company to handle classified material than a demented witch who has , on numerous occasions , mishandled classified material .",
 ['witch'],
 ['_'])

In [120]:
data[2]

('For the jobs that Lowes is laying off , there is another company taking on those jobs because the people at the store level are not doing them correctly .',
 ['level'],
 ['Rank'])

In [121]:
data[3]

("Does n't affect me personally , I am Canadian watching the US shit-show election from afar ... however the laws around impeachment in the US are very unclear .",
 ['shit-show'],
 ['_'])

In [122]:
data[4]

("Server security is an issue but it 's being used to distract from the context of the emails .",
 ['distract', 'context'],
 ['Protecting', '_'])

In [123]:
data[5]

("It 's easy to spin this to `` DNC hacked by Russia '' , and lose the original focus .",
 ['spin'],
 ['_'])

In [124]:
data[6]

('Another is a foreign government taking action for the purpose of disrupting another country for their benefit .',
 ['disrupting'],
 ['Hindering'])

In [125]:
data[7]

("`` Next year China is going to release a virus way deadlier and it 's going to cripple America because only liberal pansies will be left .",
 ['cripple'],
 ['Cause_harm'])

In [126]:
data[8]

('[ McConnell blocks democracy it every time it comes to him .',
 ['blocks'],
 ['_'])

In [127]:
data[9]

("] ( https : //www.youtube.com/watch ? time_continue=1 & amp ; v=TfQij4aQq1k ) Anytime people start jabbering about `` receipts '' , I only think of Avenatti and how much confidence this sub had on him and `` he definitely has the receipts ! '' daily claim on here .",
 ['claim'],
 ['Statement'])

In [128]:
data[10]

('The deep state is real .', ['deep'], ['_'])